## 使用生成器创建新的迭代模式

In [8]:
def frange(start, stop, increment):
    x = start
    while x < stop:
        yield x
        x += increment

In [9]:
for x in frange(0, 5, 0.4):
    print(x)

0
0.4
0.8
1.2000000000000002
1.6
2.0
2.4
2.8
3.1999999999999997
3.5999999999999996
3.9999999999999996
4.3999999999999995
4.8


In [1]:
# a little tricky with send method
def double_inputs():
    while True:
        x = yield
        yield x * 2
        yield x ** 2

In [16]:
g = double_inputs()

In [17]:
next(g)

In [20]:
g.send(30)

In [22]:
g.send(20)

400

In [88]:
def squares(start, stop):
    for i in range(start, stop):
        yield i * i

In [108]:
g = squares(1, 5)

In [110]:
g.send(14)

4

In [113]:
next(g, 'no more')

'no more'

In [121]:
class SquaresIter():
    def __init__(self, start, stop):
        self.start = start
        self.stop = stop
    def __iter__(self): return self
    def __next__(self):
        if self.start >= self.stop:
            raise StopIteration
        current = self.start * self.start
        self.start += 1
        return current

In [122]:
t = SquaresIter(10,15)

In [124]:
a = iter(t)

In [130]:
next(a)

StopIteration: 

## 实现迭代器协议

In [44]:
# 深度优先遍历树节点->中序遍历
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return f"Node({self._value})"
    
    def add_child(self, node):
        self._children.append(node)
    
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first()

In [45]:
root = Node(0)
child1 = Node(1)
child2 = Node(2)
root.add_child(child1)
root.add_child(child2)
child1.add_child(Node(3))
child1.add_child(Node(4))
child2.add_child(Node(5))

for ch in root.depth_first():
    print(ch)

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


## 反向迭代
反向迭代器非常高效，因为不需要将数据填充到一个列表然后对这个列表进行反向迭代

对于有很多数据的迭代器，转换为列表需要消耗大量内存

In [46]:
class CoundDown:
    def __init__(self, start):
        self.start = start
        
    def __iter__(self):
        n = self.start
        while n > 0:
            yield n
            n -= 1
    
    def __reversed__(self):
        n = 1 
        while n <= self.start:
            yield n
            n += 1

In [47]:
for n in CoundDown(10):
    print(n)

10
9
8
7
6
5
4
3
2
1


In [48]:
for n in reversed(CoundDown(10)):
    print(n)

1
2
3
4
5
6
7
8
9
10


## 带有外部状态的生成器函数

In [49]:
from collections import deque

In [62]:
class LineHistory:
    def __init__(self, line, histlen=3):
        self.lines = line
        self.history = deque(maxlen=histlen)
    
    def __iter__(self):
        for lineno, line in enumerate(self.lines, 1):
            self.history.append((lineno, line))
            yield line
            
    def clear(self):
        self.history.clear()

In [63]:
with open('test_file.txt', encoding='utf-8') as f:
    lines = LineHistory(f)
    for line in lines:
        if 'python' in line:
            for lineno, hline in lines.history:
                print(f"{lineno}: {hline}", end='')

1: i love python
1: i love python
2: python love me
5: where amazing happens
6: are you ready to get some
7: come on it is very nice to drink python
6: are you ready to get some
7: come on it is very nice to drink python
8: iterator is very powerful python


## 跳过可迭代对象的开始部分

In [64]:
from itertools import dropwhile

In [65]:
with open('test_file.txt', encoding='utf-8') as f:
    for line in dropwhile(lambda line: line.startswith('#'), f):
        print(line, end='')

i love python
python love me
streamlit hello world
# 我爱北京天安门
matplotlib is good
where amazing happens
are you ready to get some
come on it is very nice to drink python
iterator is very powerful python
generator is more powerful
hi seeker.

In [66]:
from contextlib import contextmanager

In [70]:
@contextmanager
def test():
    print('begin to do something')
    yield 
    print('something done, now exit')

In [71]:
with test():
    print('hello')

begin to do something
hello
something done, now exit


## 同时迭代多个对象

In [72]:
a = [1,2,3]
b = ['a','b','c','d']
list(zip(a,b))

[(1, 'a'), (2, 'b'), (3, 'c')]

In [75]:
from itertools import zip_longest

In [76]:
a = [1,2,3]
b = ['a','b','c','d']
list(zip_longest(a,b))

[(1, 'a'), (2, 'b'), (3, 'c'), (None, 'd')]

In [77]:
a = [1,2,3]
b = ['a','b','c','d']
c = ['A','B','C','D','E']
list(zip(a,b,c))

[(1, 'a', 'A'), (2, 'b', 'B'), (3, 'c', 'C')]

In [142]:
def longest_begin_substring(strs: list):
    result = ''
    for temp in zip(*strs):
        if len(set(temp)) != 1:
            break
        result += temp[0]
    return result

In [143]:
words = ['flower', 'fldow', 'flow', 'flowera']
longest_begin_substring(words)

'fl'

In [144]:
words = ['flower', 'flow', 'flow', 'flowera']
longest_begin_substring(words)

'flow'

## 不同集合上元素的迭代

In [150]:
a = (x for x in range(4))
b = (x for x in 'abcd')

In [152]:
from itertools import chain

In [153]:
for x in chain(a, b):
    print(x)

0
1
2
3
a
b
c
d


## 创建数据处理管道

In [155]:
import os

In [160]:
for path, dirlist, filelist in os.walk('../../Cookbook'):
    for file in filelist:
        if file.endswith('.ipynb'):
            print(file)

10_删除序列相同的元素并保持顺序.ipynb
13_通过关键字排序字典列表.ipynb
14-16_排序不支持原生比较的对象.ipynb
18_命名元组.ipynb
2_解压可迭代对象并赋值.ipynb
3_保留最后N个元素.ipynb
4_查找最大或最小的N个元素.ipynb
7_字典排序.ipynb
10_删除序列相同的元素并保持顺序-checkpoint.ipynb
13_通过关键字排序字典列表-checkpoint.ipynb
14-16_排序不支持原生比较的对象-checkpoint.ipynb
18_命名元组-checkpoint.ipynb
2_解压可迭代对象并赋值-checkpoint.ipynb
3_保留最后N个元素-checkpoint.ipynb
4_查找最大或最小的N个元素-checkpoint.ipynb
7_字典排序-checkpoint.ipynb
1_使用多个界定符分割字符串.ipynb
1_使用多个界定符分割字符串-checkpoint.ipynb
1_手动遍历迭代器.ipynb
2_代理迭代.ipynb
3.ipynb
1_手动遍历迭代器-checkpoint.ipynb
2_代理迭代-checkpoint.ipynb
3-checkpoint.ipynb
2_只接受关键字参数的函数.ipynb
3_增加函数元信息.ipynb
5_定义有默认参数的函数.ipynb
7_匿名函数捕获变量值.ipynb
8_偏函数减少可调用对象参数个数.ipynb
2_只接受关键字参数的函数-checkpoint.ipynb
3_增加函数元信息-checkpoint.ipynb
5_定义有默认参数的函数-checkpoint.ipynb
7_匿名函数捕获变量值-checkpoint.ipynb
8_偏函数减少可调用对象参数个数-checkpoint.ipynb
7_调用父类方法.ipynb
Untitled.ipynb
7_调用父类方法-checkpoint.ipynb
Untitled-checkpoint.ipynb
22_定义上下文管理器.ipynb
22_定义上下文管理器-checkpoint.ipynb


## 展开嵌套的序列

In [161]:
items = [1,2,[3,4,[5,6],7],8]

In [162]:
len(items)

4

In [171]:
from collections import Iterable
def flatten(items, ignore_types=(str, bytes)):
    for i in items:
        if isinstance(i, Iterable) and not isinstance(i, ignore_types):
#             method 1
            yield from flatten(i)
#             method 2
#             for i in flatten(i):
#                 yield i
        else:
            yield i
list(flatten(items))

[1, 2, 3, 4, 5, 6, 7, 8]

## 迭代器代替while无限循环

In [182]:
import sys
s = open('test_file.txt', encoding='utf-8')
# iter接受可选callable对象以及标记值作为输入参数-->创建迭代器直到callable返回值与标记值相等
for chunk in iter(lambda: s.read(1), 'n'):
    n = sys.stdout.write(chunk)
    print('--')

#--
 --
没--
有--
什--
么--
了--
不--
起--
的--

--
#--
 --
测--
试--
文--
件--

--
i--
 --
l--
o--
v--
e--
 --
p--
y--
t--
h--
o--
